In [35]:
import pandas as pd
from numpy.random import RandomState
from itertools import product

######## FUNCTIONS
#----------------------
def simple_shuffle(frame, block=None, times=10, seed=None, resetInd=False):
    """
    Shuffles trials a few times.
    
    :param pandas.DataFrame frame: Trials to be shuffled.
    :param block: Optional column to groupby before shuffling.
    :type block: str or None.
    :param int times: Number of times to shuffle. Defaults to 10.
    :param seed: Seed random number generator.
    :type seed: int or None
    :returns: Trial list with rows in random order.
    :rtype: pandas.DataFrame
    """
    prng = RandomState(seed)
    
    def _shuffle(chunk):
        for _ in range(times):
            chunk = chunk.reindex(prng.permutation(chunk.index))
        if resetInd is True:
            chunk=chunk.reset_index(drop=True)
        return chunk
    
    if block is None:
        return _shuffle(frame)
    else:
        return frame.groupby(block).apply(_shuffle)
    
def add_block(frame, size, name='block', start_at=0, id_col=None, seed=None):
    """
    Creates a new column for block.
    
    :param pandas.DataFrame frame: Trials to be assigned blocks.
    :param int size: Length of each block.
    :param id_col: Column to group by before blocking. Assures that blocks 
        consist of approximately the same number of trials for each unique
        value in id_col
    :type id_col: str or None
    :param seed: Seed random number generator.
    :type seed: int or None
    :returns: Trial list with new column for block.
    :rtype: pandas.DataFrame
    """
    def _assigner(blocks, prng):
        prng.shuffle(blocks)
        i = 0
        while True:
            yield blocks[i]
            if (i+1)%len(blocks):
                prng.shuffle(blocks)
            i = (i+1)%len(blocks)
            
    prng = RandomState(seed)
    blocks = range(len(frame)/size)
    assigner = _assigner(blocks, prng)
    
    def _add(chunk):
        chunk[name] = [assigner.next() for _ in xrange(len(chunk))]
        return chunk
    
    if id_col is None:
        new_frame = _add(frame).sort_values(by=name)
    else:
        new_frame = frame.groupby(id_col).apply(_add).sort_values(by=name)
    
    new_frame[name] = new_frame[name] + start_at
    return new_frame

def genTr(df,firstBlockSize, secondBlockSize,seedOfShuffle=None):
    ## Split halfway, assign expectedness
    block4Rel=add_block(df,firstBlockSize,name='block',start_at=0,seed=seedOfShuffle)
    relConds=pd.DataFrame({'block':[1,2],'cond':['rel','unrel']})
    relAss=pd.merge(block4Rel,relConds,how='left')
    relAss=relAss.drop('block',1)

    ## Split 4-way within-expectedness, assign speaker
    block4Speak=add_block(relAss,secondBlockSize,name='block',id_col='cond',start_at=0,seed=seedOfShuffle)
    speakAss=block4Speak.sort_values(by=['block','cond'])
    speakConds=pd.DataFrame({'block':[1,2,3,4],'speaker':['nat','nonNat','nat','nonNat']})
    speakAss=pd.merge(block4Speak,speakConds,how='left')
    speakAss=speakAss.drop('block',1)
    
    return speakAss

######## IMPLEMENTATIONS
#-------------------------   

sentTypes=pd.read_csv('./mockTrialGen/mockList.csv')
trialGen=genTr(sentTypes,firstBlockSize=60,secondBlockSize=30,seedOfShuffle=1)



In [36]:
trialGen
trialGen.to_csv('testTrialList.csv',index=False)

In [63]:
test=add_block(sentTypes,60,name='block',start_at=1,seed=2)
test=test.groupby('block')
len(test.get_group(1))

59

In [1]:
blocks = range(len(sentTypes)/60)
60%60

NameError: name 'sentTypes' is not defined

In [6]:
120%60

0